Lab 5 
Andrik Solano Jimenez Ronald Arce Matamoros

Libreríaas

In [7]:
import os
import random
import numpy as np
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


In [8]:
# Ruta de las carpetas con las imágenes
with_mask_dir = "./with_mask"
without_mask_dir = "./without_mask"

# Función para cargar y redimensionar imágenes
def load_and_resize_images(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        img = cv2.imread(os.path.join(directory, filename))
        img = cv2.resize(img, (100, 100))  # Cambia el tamaño de las imágenes según tu preferencia
        if img is not None:
            images.append(img)
            labels.append(label)
    return images, labels

# Cargar y redimensionar imágenes con máscara
with_mask_images, with_mask_labels = load_and_resize_images(with_mask_dir, 1)

# Cargar y redimensionar imágenes sin máscara
without_mask_images, without_mask_labels = load_and_resize_images(without_mask_dir, 0)

KeyboardInterrupt: 

Entendimiento de los datos

Procesamiento de los datos

In [ ]:
# Combinar imágenes y etiquetas
images = np.array(with_mask_images + without_mask_images)
labels = np.array(with_mask_labels + without_mask_labels)

# Dividir los datos en entrenamiento, validación y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Normalizar los valores de píxeles
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Convertir etiquetas a formato one-hot encoding
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [ ]:
# Crear el generador de datos de imagen para aumentación de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Ajustar el generador de datos de imagen a los datos de entrenamiento
train_datagen.fit(X_train)

Modelo Deep Learning

In [ ]:
# Compilar el modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.3),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compilar el modelo con el optimizador Adam
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Entrenar el modelo
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) / 32,
    epochs=20,
    validation_data=(X_val, y_val),
    verbose=1
)


Evaluacion

In [9]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", test_accuracy)

48/48 [==============================] - 1s 14ms/step - loss: 16.8883 - accuracy: 0.9523
Test accuracy: 0.9523494243621826


Resultados